# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

Прежде чем переходить к анализу, необходимо составить представление о данных, полученных от кредитного отдела банка. Для начала импортируем библиотеку `pandas`, прочитаем файл `data.csv` и сохраним его в переменной `df`:

In [1]:
# импорт библиотеки pandas
import pandas as pd

In [2]:
# чтение файла с данными и сохранение в df
df = pd.read_csv('/datasets/data.csv')

Выведем на экран первые десять строк таблицы: <a id='data'></a>

In [3]:
#просмотр первых десяти строк таблицы
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Далее выведем общую информацию о таблице:

In [4]:
#получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Что мы имеем:** в таблице всего 12 столбцов, в которых встречаются три разных типа данных: `int64`, `float64`, `object`.

Согласно документации к данным:
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

Нарушений стиля не выявлено, однако количество значений в таблице разлиается. Это свидетельствует о том, что в данных имеются пропуенные значения, которые будут обработаны в дальнейшем.

**Вывод**

В каждой строке таблицы содержатся сведения о заемщике. Предварительно можно резюмировать, что представленных данных достаточно для проверки гипотез. Но прежде чем это делать, необходимо осуществить операции по их предобработке. 

## Шаг 2. Предобработка данных

### Обработка пропусков

#### Поиск и обработка ожидаемых пропусков

Для поиска пропущенных значений обратимся к методу `isna()`, передав результат его работы методу `sum()`:

In [5]:
# подсчёт пропусков
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропущенные значения присутствуют в двух столбцах: `days_employed` и `total_income`. Причины появления пропусков могут быть совершенно разные: клиент попросту решил не указывать эту информацию; на момент внесения данных они не были ему известны; либо же это просто технический сбой. Однако информация в данных столбцах пригодится для исследования, поэтому нельзя полностью удалять строки, в которых обнаружены пропуски. По возможности их нужно сохранить.


Прежде чем заполнить пропуски, исследуем данные, содержащиеся в столбцах days_employed и total_income.

In [7]:
#посмотрим, какие данные содержатся в days_employed
df['days_employed'].unique()

array([-8437.67302776, -4024.80375385, -5623.42261023, ...,
       -2113.3468877 , -3112.4817052 , -1984.50758853])

In [8]:
#посмотрим, какие данные содержатся в total_income
df['total_income'].unique()

array([253875.6394526 , 112080.01410244, 145885.95229686, ...,
        89672.56115303, 244093.05050043,  82047.41889948])

Помимо пропусков столбец `days_employed` содержит отрицательные аномальные значения, которые могли появиться в силу того, что информация бралась из нескольких систем, где стаж считается в часах. Для начала приведем их к положительным и поделим на 24:

In [9]:
#отрицательные значения приведем к положительным
#переведем их в часы и
#выведем на экран уникальные значения 
df['days_employed'] = df['days_employed'].apply(abs)
df.loc[df['days_employed'] >30000, 'days_employed'] = df.loc[df['days_employed'] >30000, 'days_employed'] / 24
df['days_employed'].unique()

array([8437.67302776, 4024.80375385, 5623.42261023, ..., 2113.3468877 ,
       3112.4817052 , 1984.50758853])

Теперь можно приступить к замене пропусков. Сделаем это исходя из разных типов занятости. Найдем медианные значения рабочего стажа, а также суммарного дохода в каждом типе занятости и подставим их в соответствующие пропуски. Сперва посмотрим, какие типы занятости встречаются среди пропущенных значений в столбцах `days_employed` и `total_income`:

In [10]:
#просмотр уникальных типов занятости, в которых присутствуют пропущеные значения с трудовом стаже
df[df['days_employed'].isna()]['income_type'].unique()

array(['пенсионер', 'госслужащий', 'компаньон', 'сотрудник',
       'предприниматель'], dtype=object)

In [11]:
#просмотр уникальных типов занятости, в которых присутствуют пропущеные значения с доходе
df[df['total_income'].isna()]['income_type'].unique()

array(['пенсионер', 'госслужащий', 'компаньон', 'сотрудник',
       'предприниматель'], dtype=object)

Таковых 5: пенсионер , госслужащий, компаньон, сотрудник и предприниматель. Найдем медианные значения стажа для каждой категории.

In [12]:
#просмотр медианного стажа для каждой категории
income_type = ['пенсионер', 'госслужащий', 'компаньон', 'сотрудник', 'предприниматель']
for income in income_type:
    print(income, df[df['income_type'] == income]['days_employed'].median())

пенсионер 15217.221094405468
госслужащий 2689.3683533043886
компаньон 1547.3822226779334
сотрудник 1574.2028211070854
предприниматель 520.8480834953765


Далее найдем медианные значения для дохода в каждой категории:

In [13]:
#просмотр медианного дохода для каждой категории
income_type = ['пенсионер', 'госслужащий', 'компаньон', 'сотрудник', 'предприниматель']
for income in income_type:
    print(income, df[df['income_type'] == income]['total_income'].median())

пенсионер 118514.48641164352
госслужащий 150447.9352830068
компаньон 172357.95096577113
сотрудник 142594.39684740017
предприниматель 499163.1449470857


Объединим все в один цикл и заполним пропуски в соответствии с типом занятости:

In [14]:
#подставим медианное значение каждой категории в столбцы days_employed и total_income
income_type = ['пенсионер', 'госслужащий', 'компаньон', 'сотрудник', 'предприниматель']
for income in income_type:
    df.loc[(df['days_employed'].isna()) & (df['income_type'] == income), 'days_employed'] = df[df['income_type'] == income]['days_employed'].median()
    df.loc[(df['total_income'].isna()) & (df['income_type'] == income), 'total_income'] = df[df['income_type'] == income]['total_income'].median()

Проверим, остались ли пропуски:

In [15]:
# подсчёт пропусков
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Пропуски в столбцах `days_employed` и `total_income` успешно заполнены медианными значениями.

#### Поиск и обработка странных пропусков

Пропуски, которые были заполнены, относятся к категории ожидаемых. Однако в представленном наборе данных могут присутствовать, так назыываемые, странные пропущенные значения. На первый взгляд может показаться, что все заполнено правильно, и можно приступать к анализу. Однако необходимо изучить таблицу на предмет очевидно неверных и странных значений с целью их замены. Для этого с помощью метода `value_counts` можно получить уникальные значения, содержащиеся в каждом столбце, а затем найти выбивающиеся из общего списка: <a id='unique'></a>

In [16]:
#просмотр уникальных значений в каждом столбце таблицы
for column in df.columns:
    print('----------------------------')
    print(column)
    print(df[column].value_counts())

----------------------------
children
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
----------------------------
days_employed
1574.202821     1105
1547.382223      509
15217.221094     414
2689.368353      147
520.848083         2
                ... 
2484.080972        1
595.521199         1
1683.013673        1
2585.402858        1
1636.419775        1
Name: days_employed, Length: 19353, dtype: int64
----------------------------
dob_years
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    1

Итак, в результате можно заметить следующие странные значения:
* столбец `children` - значения `-1` и `20` выбиваются из общего списка. Можно предположить, что `-1` означает ситуацию, когда детей нет, а `20` - опечатка, поскольку цифры `2` и `0` находятся рядом;
* столбец `dob_years` - 101 раз встречается значение `0`. Скорее всего пользователи принебрегли заполнением данного поля, а система решила сделать это за них;
* столбец `gender` - непонятное значение `XNA`, в дальнейше заменим его на наиболее часто встречающееся в данном столбце, поскольку оно повторяется всего лишь один раз, и данная замена не окажется критичной

**Столбец `children`**

Вместо `-1` поставим `0`, а вместо `20` - `2` с помощью `replace()`:

In [17]:
#замена значений в столбце children и 
#вывод на экран уникальных значений 
df['children'] = df['children'].replace(20, 2)
df['children'] = df['children'].replace(-1, 0)
df['children'].unique()

array([1, 0, 3, 2, 4, 5])

Столбец с данными о количестве детей исправлен. Можно заметить, что среди заемщиков преобладают люди, не имеющие детей. Однако также есть и многодетные.

**Столбец `dob_years`**

Обработаем значение `0` как пропуск, а именно заменим его медианным значением:


Выведем строки, в которых возраст клиента равен 0: 

In [19]:
df[df['dob_years'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,14439.234121,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,16577.356876,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,1574.202821,0,среднее,1,женат / замужем,0,F,сотрудник,0,142594.396847,жилье
20462,0,14113.952856,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,покупка своего жилья
20577,0,13822.552977,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,недвижимость
21179,2,108.967042,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости


Посмотрим, на какие категории занятости приходятся нулевые значения возраста:

In [20]:
df[df['dob_years'] == 0]['income_type'].unique()

array(['пенсионер', 'сотрудник', 'компаньон', 'госслужащий'], dtype=object)

Заполним нулевые значения в зависимости от типа занятости:

In [21]:
#просмотр медианного возраста для каждой категории
income_type = ['пенсионер', 'госслужащий', 'компаньон', 'сотрудник']
for income in income_type:
    print(income, df[df['income_type'] == income]['dob_years'].median())

пенсионер 60.0
госслужащий 40.0
компаньон 39.0
сотрудник 39.0


In [22]:
#подставим медианное значение каждой категории в столбец dob_years
income_type = ['пенсионер', 'госслужащий', 'компаньон', 'сотрудник']
for income in income_type:
    df.loc[(df['dob_years'] == 0) & (df['income_type'] == income), 'dob_years'] = df[df['income_type'] == income]['dob_years'].median()

Проверим, остались ли в датафрейме строки, в которых возраст был бы равен нулю:

In [23]:
df[df['dob_years'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Строки, в которых возраст равен нулю, отсутствуют.

**Столбец `gender`**

Заменим значение `XNA` на наиболее встречающееся в данном столбце - `F` с помощью `replace()`:

In [24]:
#замена данных в столбце gender и 
#вывод на экран уникальных значений 
df['gender'] = df['gender'].replace('XNA', 'F')
df['gender'].unique()

array(['F', 'M'], dtype=object)

Столбец с данными о половой принадлежности клиентов исправлен.

**Вывод**

При обработке данных были обнаружены ожидаемые и странные пропуски. В силу того, что строками, содержащими пропущенные значениями, нельзя было пренебречь, было принято решение об их замене. Для этого были использваны `loc` и `replace()`. По итогу потери в данных не являются критичными, а конечная информация, содержащаяся в таблице, пригодна для дальнейшего анализа.

### Замена типа данных

После проведенных выше изменений необходимо еще раз вывести общую информацию о таблице:

In [25]:
#получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  float64
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


Pandas посчитал данные о трудовом стаже, возрасте и общем доходе вещественными. Выведем еще раз первые 10 строк таблицы, чтобы посмотреть, как это выглядит:

In [26]:
#просмотр первых десяти строк таблицы
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50.0,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35.0,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Что получается на выходе:
* В столбце `dob_years` c точки зрения Python, результат — вещественное число. Но фактически это число целое — ведь у него нет дробной части.
* В столбце `days_employed` присутствует дробная часть. Это выглядит странно, поскольку принято считать трудовой стаж в виде полных дней.
* В столбце `total_income` также есть дробная часть. Однако нам в дальнейшем не потребуются математические расчеты, где необходимо учитывать сумму до последней копейки, поэтому данные в этом столбце также можно привести к целочисленным.

Необходимо преобразовать тип данных в перечисленных выше столбцах: перевести значение из вещественного типа в целочисленный, из `float` в `int`. Для этого лучше всего подойдет метод `astype()`:

In [27]:
#напишем цикл, который переберет нужные столбцы и
#выведем на экран общую информацию о таблице
columns = ['days_employed', 'dob_years', 'total_income']
for column in columns:
    df[column] = df[column].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

С помощью метода `astype()` была произведена замена типов данных в столбцах `days_employed`, `dob_years`, `total_income` с `float` на `int`. Данный метод был выбран потому, что он позволяет быстро перевести значение в нужный тип.  

### Обработка дубликатов

Ранее при выводе [первых десяти строк таблицы](#data) в столбце `education` были замечены значения, отличающиеся регистром. Их появление вполне объяснимо и скорее всего напрямую зависит от клиента: либо у него случайно была зажата клавиша CapsLock, либо он ввел так намеренно. 

Необходимо проверить какие уникальные записи содержатся в столбце `education`. Для этого подойдет метод `unique()`:

In [28]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Каждый уровень образования дублируется трижды! Приведем данные к единообразному виду с помощью метода `str.lower()`:

In [29]:
#замена значений в столбце education на нижний регистр и
#вывод количества уникальных записей
df['education'] = df['education'].str.lower()
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

В столбце `education` все данные записаны строчными буквами. Это выглядит единообрано.

Теперь проверим, есть ли в таблице строки, полностью дублирующие друг друга. Для этого используем `loc`:

In [30]:
#keep = False выведет все совпадения, таким способом можно получить полные дубликаты
df[df.duplicated(keep = False)].sort_values(by = list(df.columns))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
9374,0,1547,38,высшее,0,гражданский брак,1,F,компаньон,0,172357,на проведение свадьбы
19387,0,1547,38,высшее,0,гражданский брак,1,F,компаньон,0,172357,на проведение свадьбы
5124,0,1547,40,среднее,1,гражданский брак,1,F,компаньон,0,172357,сыграть свадьбу
10697,0,1547,40,среднее,1,гражданский брак,1,F,компаньон,0,172357,сыграть свадьбу
16148,0,1547,45,среднее,1,гражданский брак,1,F,компаньон,0,172357,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...
9238,2,1574,34,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для сдачи
11033,2,1574,39,среднее,1,гражданский брак,1,F,сотрудник,0,142594,сыграть свадьбу
16902,2,1574,39,среднее,1,гражданский брак,1,F,сотрудник,0,142594,сыграть свадьбу
9013,2,2689,36,высшее,0,женат / замужем,0,F,госслужащий,0,150447,получение образования


137 строк, полностью дублирующих друг друга. О причинах их появления можно только догадываться: возможно пользователь дважды заполнил форму; либо произошел технический сбой; или данные получены из нескольких систем, где могла содержаться одна и та же информация.

Прежде чем удалять дубликаты, для интереса посчитаем количество строк в таблице. Используем для этого метод `shape`:

In [31]:
#подсчет количества строк в таблице
print('Количество строк до удаления:', df.shape)

Количество строк до удаления: (21525, 12)


Удалять дубликаты необходимо с помощью метода `drop_duplicates()`:

In [32]:
#удаление дубликатов и 
#подсчет количества строк
df = df.drop_duplicates()
print('Количество строк после удаления:', df.shape)

Количество строк после удаления: (21454, 12)


Удалена 71 строка. Необходимо проверить, остались ли еще дублирующие друг друга строки:

In [33]:
#вывод полных дубликатов
df[df.duplicated(keep = False)].sort_values(by = list(df.columns))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Строки, полность дублирующие друг друга, удалены. 

**Вывод**

С помощью метода `str.lower()` данные столбца `education` были приведены к нижнему регистру. Таке был применен метод `drop_duplicates()`, благодаря которому были удалены полные дубликаты, которые в дальнейшем могли задваивать информацию и отрицательным образом влиять на результаты исследования.

### Лемматизация

[Ранее при просмотре уникальных значений каждого столбца](#unique) было замечено, что в столбце `purpose` содержатся данные, по большей части дублирующие друг друга: некоторые значения с одинаковым смыслом записаны по-разному. Для того, чтобы привести их все к одной форме, необходимо обратиться к процессу лемматизации. Выведем еще раз уникальные значения столбца `purpose`:

In [34]:
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Как можно заметить, в результирующей таблице присутствуют значения "свадьба", "на проведение свадьбы", "сыграть свадьбу", что по сути является одним и тем же. И в сумме данная цель должна повторяться 2324 раза. И это не единственная цель с подобной проблемой. Вероятнее всего различия в формулировке - чисто человеческий фактор: как клиент указал, так оно и было внесено в таблицу. Такой особенности можно было бы избежать, предоставив человеку определенный набор уже сформулированных целей, однако в таком случае сложно продумать все возможные варианты, а возможность самостоятельного ввода цели дало бы точно таккой же результат, который имеется на данный момент.

Для того, чтобы проводить дальнейший анализ необходимо привести данные к единообразию. В этом поможет библиотека с функцией лемматизации `pymystem3`. Импортируем данную библиотеку и выведем на экран леммы слов, содержащихся в столбце `purpose`:

In [35]:
# импорт pymystem3
#вывод на экран лемм 
from pymystem3 import Mystem
m = Mystem() 
lemmas = m.lemmatize(' '.join(df['purpose'].unique()))
lemmas

['покупка',
 ' ',
 'жилье',
 ' ',
 'приобретение',
 ' ',
 'автомобиль',
 ' ',
 'дополнительный',
 ' ',
 'образование',
 ' ',
 'сыграть',
 ' ',
 'свадьба',
 ' ',
 'операция',
 ' ',
 'с',
 ' ',
 'жилье',
 ' ',
 'образование',
 ' ',
 'на',
 ' ',
 'проведение',
 ' ',
 'свадьба',
 ' ',
 'покупка',
 ' ',
 'жилье',
 ' ',
 'для',
 ' ',
 'семья',
 ' ',
 'покупка',
 ' ',
 'недвижимость',
 ' ',
 'покупка',
 ' ',
 'коммерческий',
 ' ',
 'недвижимость',
 ' ',
 'покупка',
 ' ',
 'жилой',
 ' ',
 'недвижимость',
 ' ',
 'строительство',
 ' ',
 'собственный',
 ' ',
 'недвижимость',
 ' ',
 'недвижимость',
 ' ',
 'строительство',
 ' ',
 'недвижимость',
 ' ',
 'на',
 ' ',
 'покупка',
 ' ',
 'подержать',
 ' ',
 'автомобиль',
 ' ',
 'на',
 ' ',
 'покупка',
 ' ',
 'свой',
 ' ',
 'автомобиль',
 ' ',
 'операция',
 ' ',
 'с',
 ' ',
 'коммерческий',
 ' ',
 'недвижимость',
 ' ',
 'строительство',
 ' ',
 'жилой',
 ' ',
 'недвижимость',
 ' ',
 'жилье',
 ' ',
 'операция',
 ' ',
 'со',
 ' ',
 'свой',
 ' ',
 'недвижимо

Для того, чтобы сделать столбец `purpose` единообразным, необходимо посчитать частоту выявления лемм с помощью контейнера `Counter`, импортировав `collections`, посмотреть, какие цели встречаются чаще всего, оставить их, а вместо остальных поставить `прочее`.

In [36]:
#подсчёт числа упоминаний лемматизированных слов
from collections import Counter
Counter(lemmas)

Counter({'покупка': 10,
         ' ': 96,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

Наиболее часто встречающиеся лемматизированные цели: автомобиль, образование, недвижимость, жилье, свадьба. Проведем лемматизацию целей кредита:

In [37]:
#создание столбца purpose_lemm, который будет принимать значение леммы
#просмотр первых двадцати строк таблицы
purposes = ['недвижимость', 'автомобиль', 'образование', 'свадьба', 'жилье']
def purpose_lemm(value): 
    lemmas = m.lemmatize(value)
    for purpose in purposes:
        if purpose in lemmas:
            if purposes[-1] in lemmas:
                return purposes[0]
            return purpose
    return 'прочее'
df['purpose_lemm'] = df['purpose'].apply(purpose_lemm)
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemm
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


В новом столбце `purpose_lemm` можно увидеть лемматизированные значения целей.

**Вывод**

Благодаря лемматизации удалось преобразовать слова, содержащиеся в столбце `purpose`, в их значимые базовые формы, поскольку данный процесс учитывает контекст. В дальнейшем при проведении анализа будет удобно категоризировать данные для получения определенных выводов.

### Категоризация данных

В заключительной части проекта стоит задача выявить зависимость между наличием детей, семейным положением, уровнем дохода, цели кредита и его вовращением в срок. Для этого необходимо узнать, какие уникальные значения содержит каждая категориальная переменная.

#### Категоризация по семейному положению

Рассмотрим данные о семейном положении: создадим "словарь", где названию категории будет соответствовать номер. В итоге получим данные с семейным статусом клиентов:

In [38]:
family_dict = df[['family_status_id', 'family_status']].drop_duplicates().set_index('family_status_id')
family_dict

,family_status
family_status_id,
0,женат / замужем
1,гражданский брак
2,вдовец / вдова
3,в разводе
4,Не женат / не замужем


В таблице присутствует 5 категорий клиентов. Каждой категории соответствует свой уникальный идентификатор.

#### Категоризация по доходу

Выведем справочно минимальный, максимальный и медианный уровень дохода клиентов, используя `min()`, `max()`, `median()`:

In [39]:
print('Минимальный доход:', df['total_income'].min())
print('Максимальный доход:', df['total_income'].max())
print('Медианное значение:', int(df['total_income'].median()))

Минимальный доход: 20667
Максимальный доход: 2265604
Медианное значение: 142594


Пороговые значения известны, исходя из них можно сформировать следующие категории:
* Клиенты с доходом ниже 80 тыс.руб. попадают в категорию "низкий";
* Клиенты с доходом от 81 тыс.руб. до 145 тыс.руб. попадают в категорию "ниже среднего";
* Клиенты с доходом от 146 тыс.руб. до 165 тыс.руб. попадают в категорию "средний";
* Клиенты с доходом от 166 тыс.руб. до 200 тыс.руб. попадают в категорию "выше среднего";
* Клиенты с доходом 201 тыс.руб. и выше попадают в категорию "высокий".

Далее необходимо написать функцию, которая вернет категорию клиенту в зависимости от суммы его дохода. После, с помощью `apply` создадим столбец, в который будет передано ее значение и вернем первые 10 строк таблицы:

In [40]:
def income_group(income):
    if income <= 80000:
        return 'низкий'
    if income <= 145000:
        return 'ниже среднего'
    if income <= 165000:
        return 'средний'
    if income <= 200000:
        return 'выше среднего'
    return 'высокий'
df['income_group'] = df['total_income'].apply(income_group)
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemm,income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,ниже среднего
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,высокий
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,средний
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,высокий
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,высокий
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,ниже среднего
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,ниже среднего
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,ниже среднего


Выведем статистику по уровню дохода клиентов банка методом `value_counts()`:

In [41]:
#просмотр уникальных значений столбца income_group
df['income_group'].value_counts()

ниже среднего    8854
высокий          5067
выше среднего    3142
низкий           2276
средний          2115
Name: income_group, dtype: int64

Полученные данные пригодятся для проверки одной из гипотез.

#### Категоризация по целям кредита

Ранее с помощью лемматизации были выделены основные цели, на которые клиенты хотят взять кредит: недвижимость, жилье, автомобиль, образование, свадьба:

In [42]:
#просмотр уникальных значений столбца purpose_lemm
df['purpose_lemm'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_lemm, dtype: int64

По полученным данным можно определить, какая цель пользуется большей популярностью среди заемщиков (недвижимость), а какая меньше (свадьба). Осталось выяснить, влияет ли это каким-то образом на возврат кредита в срок.

#### Категоризация по наличию детей

Изучим содержимое столбца `children`:

In [43]:
#просмотр уникальных значений столбца children
df['children'].value_counts()

0    14138
1     4808
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Среди клиентов банка присутствуют те, у кого нет детей, у кого 1-2 ребенка, а также есть многодетные. Так как вопрос поставлен следующим образом: "Есть ли зависимость между наличием детей и возвратом кредита в срок?" , целесообразно всех заемщиков разделить на следующие категории:
* Клиенты, у которых 0 детей, попадают в категорию "нет детей";
* Клиенты, у которых 1-5 детей, попадают в категорию "дети есть".

Напишем функцию, которая вернет категорию клиенту в зависимости от наличия детей. После с помощью `apply()` создадим столбец, в который будет передано ее значение и вернем первые 10 строк таблицы:

In [44]:
def children_group(children):
    if children == 0:
        return 'нет детей'
    return 'дети есть'
df['children_group'] = df['children'].apply(children_group)
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemm,income_group,children_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,высокий,дети есть
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,ниже среднего,дети есть
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,средний,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,высокий,дети есть
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,средний,нет детей
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,высокий,нет детей
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,высокий,нет детей
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,ниже среднего,нет детей
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,ниже среднего,дети есть
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,ниже среднего,нет детей


Выведем статистику по количеству детей клиентов банка методом `value_counts()`:

In [45]:
#просмотр уникальных значений столбца income_group
df['children_group'].value_counts()

нет детей    14138
дети есть     7316
Name: children_group, dtype: int64

Клиентов, у которых нет детей, в два раза больше. Какая из категорий наиболее платежеспособна, узнаем далее.

**Вывод**

Прежде чем приступить к определению влияния различных факторов на возвратность кредита в срок, данные были категоризированы. Данный шаг необходим, поскольку без него работа с таблицей в следующем разделе значительно усложнилась бы.

## Шаг 3. Ответьте на вопросы

Для того, чобы ответить на поставленные ниже вопросы, необходимо будет построить сводные таблицы. В строках должны быть категории заемщиков, в столбцах отсутствие или наличие долга, на пересечении - число заявок на кредит. Отдельным столбцом следует вынести процент невозврата кредита в срок - это и будет являться основной метрикой для поиска наиболее надежной категории клиентов.

Сводные таблицы будем создавать с помощью метода `pivot_table()`.

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [46]:
#создадим сводную таблицу
chidren_report = df.pivot_table(index = 'children_group', columns = 'debt', values = 'dob_years', aggfunc = 'count')
chidren_report.columns = ['no_debt', 'debt']
chidren_report['sum'] = chidren_report['debt'] + chidren_report['no_debt']
chidren_report['%'] = (chidren_report['debt']/chidren_report['sum']) * 100
round(chidren_report, 2)


,no_debt,debt,sum,%
children_group,,,,
дети есть,6639,677,7316,9.25
нет детей,13074,1064,14138,7.53


**Вывод**

За неимением данных о суммах кредитов, не представляется возможным смоделировать потенциальные убытки от каждой категории. Поэтому в таком случае логично сообщить, у кого процент больше, а у кого меньше. Процент кредитной задолженности у людей с детьми выше, следовательно в данной категории более платежеспособная группа - клиенты, не имеющие детей. Получается, что зависимость есть: наличие детей отрицательно влияет на возврат кредита.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [47]:
#создадим сводную таблицу
family_report = df.pivot_table(index = 'family_status', columns = 'debt', values = 'dob_years', aggfunc = 'count')
family_report.columns = ['no_debt', 'debt']
family_report['sum'] = family_report['debt'] + family_report['no_debt']
family_report['%'] = (family_report['debt']/family_report['sum']) * 100
round(family_report, 2)


,no_debt,debt,sum,%
family_status,,,,
Не женат / не замужем,2536,274,2810,9.75
в разводе,1110,85,1195,7.11
вдовец / вдова,896,63,959,6.57
гражданский брак,3763,388,4151,9.35
женат / замужем,11408,931,12339,7.55


**Вывод**

Лучше всего выплачивают кредит овдовевшие клиенты и люди в разводе. Не далеко от них ушли люди семейные. Хуже - не соедненные узами брака, а также состоящие в гражданском браке. Выходит, что зависимость между семейным положением и возвратом кредита в срок отсутствует: и одинокие, и женатые/замужние относительно одинаково погашают задолженность.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [48]:
income_report = df.pivot_table(index = 'income_group', columns = 'debt', values = 'dob_years', aggfunc = 'count')
income_report.columns = ['no_debt', 'debt']
income_report['sum'] = income_report['debt'] + income_report['no_debt']
income_report['%'] = (income_report['debt']/income_report['sum']) * 100
round(income_report, 2)


,no_debt,debt,sum,%
income_group,,,,
высокий,4709,358,5067,7.07
выше среднего,2870,272,3142,8.66
ниже среднего,8091,763,8854,8.62
низкий,2102,174,2276,7.64
средний,1941,174,2115,8.23


**Вывод**

Вполне ожидаемо, что люди с высоким уровнем дохода яляются наиболее платежеспособной группой. Однако тот факт, что люди, имеющие доход выше среднего имеют больше задолженности, чем люди с низким уровнем дохода, говорит о том, что зависимости нет.



- Как разные цели кредита влияют на его возврат в срок?

In [49]:
purpose_report = df.pivot_table(index = 'purpose_lemm', columns = 'debt', values = 'dob_years', aggfunc = 'count')
purpose_report.columns = ['no_debt', 'debt']
purpose_report['sum'] = purpose_report['no_debt'] + purpose_report['debt']
purpose_report['%'] = (purpose_report['debt']/purpose_report['sum']) * 100
round(purpose_report, 2)

,no_debt,debt,sum,%
purpose_lemm,,,,
автомобиль,3903,403,4306,9.36
недвижимость,10029,782,10811,7.23
образование,3643,370,4013,9.22
свадьба,2138,186,2324,8.00


**Вывод**

Люди, берущие кредит на недвижимость - самые платежеспособные клиенты. В очереди за ними находятся те, кто имеет кредит на проведение свадебного торжества. Больше всего должников среди тех, кто взял кредит на автомобиль и на образование. 

## Шаг 4. Общий вывод

В начале исследования стояла задача выяснить, влияет ли наличие детей, а также семейное положение клиентов на погашение кредита в срок. 

Влияет ли семейное положение на факт погашения кредита в срок? Четкой зависимости нет: взвратность кредита среди семейных, а также одиноких людей находится примерно на одном уровне.

Влияет ли количество детей клиента на факт погашения кредита в срок? По результатам анализа более платежеспособной группой оказались люди, не имеющие детей. 